In [1]:
import numpy as np
from matplotlib import pyplot as plt
# TODO: add torch

In [2]:
toy_A = np.array([[3,2],[2,6]])
toy_b = np.array([[2],[-8]])

In [26]:
def steepest_descent(A, b, x0, max_iter=int(1e5), tol=1e-5, recompute=None):
    """
    Solving Ax = b or min(0.5xTAx - bTx + c)
    Steepest descent is at orthogonal to the previous one in each iteration.
    Suitable for scipy sparse and numpy dense arrays.
    Using fast iterative update of residual, rounding errors accumulates.

    A: dense or sparse matrix
    b, x0: column vectors
    recompute: number of iterations to recompute the exact residual, infered if not given

    returns:
        x: final x
        r: final residual
        num_iter: number of iterations run
    """
    if recompute is None:
        recompute = np.sqrt(A.shape[0])
    recompute = int(recompute)

    x = x0
    r = b - A@x
    delta = r.T@r
    tolerance = tol**2 * delta # when ||r_i|| <= tol * ||r_0||
    num_iter = max_iter
    for i in range(max_iter):
        q = A@r
        alpha = delta / (r.T@q)
        x = x + alpha *r
        if i % recompute == 0:
            r = b - A@x
        else:
            r = r - alpha*q
        delta = r.T@r

        if delta <= tolerance:
            num_iter = i+1
            break
    return x, r, num_iter

In [31]:
x0 = np.array([[-1],[-2]])
x, r, num = steepest_descent(toy_A, toy_b, x0, tol=1e-9)
print(num, " iterations taken to get result ", x.T, " with residual ", r.T)

30  iterations taken to get result  [[ 2. -2.]]  with residual  [[7.84016407e-09 5.22677546e-09]]


In [28]:
def CG(A, b, x0, max_iter=int(1e5), tol=1e-5, recompute=None):
    """
    Solving Ax = b or min(0.5xTAx - bTx + c)
    Conjugated gradient starts with the first descent direction as d0,
    The rest of the directions are all mutually A-orthogonal and can be built from d0,
    Thus, one direction only need to descend once
    Suitable for scipy sparse and numpy dense arrays.
    Using fast iterative update of residual, rounding errors accumulates.

    A: dense or sparse matrix
    b, x0: column vectors
    recompute: number of iterations to recompute the exact residual, infered if not given

    returns:
        x: final x
        r: final residual
        num_iter: number of iterations run
    """
    if recompute is None:
        recompute = np.sqrt(A.shape[0])
    recompute = int(recompute)

    x = x0
    r = b - A@x
    d = r
    delta_new = r.T@r
    tolerance = tol**2 * delta_new # when ||r_i|| <= tol * ||r_0||
    num_iter = max_iter
    for i in range(max_iter):
        q = A@d
        alpha = delta_new / (d.T@q)
        x = x + alpha * d
        if i % recompute == 0:
            r = b - A@x
        else:
            r = r - alpha*q

        delta_old = delta_new
        delta_new = r.T@r
        beta = delta_new/delta_old
        d = r + beta*d
        
        if delta_new <= tolerance:
            num_iter = i+1
            break
    return x, r, num_iter

In [32]:
x0 = np.array([[-1],[-2]])
x, r, num = CG(toy_A, toy_b, x0, tol=1e-9)
print(num, " iterations taken to get result ", x.T, " with residual ", r.T)

2  iterations taken to get result  [[ 2. -2.]]  with residual  [[0. 0.]]
